In [2]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from Kildekode._12_IIRFilt import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Z-transformasjon

* **Emne IELEA2302 - Signalbehandling**
* **Uke 14, 2021**
* **Relevant Pensum:**
    * Kapittel 10.1-10.2 i læreboka DSP First
* **Underviser: Kai Erik Hoff**

## Tema:
* Repetisjon IIR-filtre
* Fra differanseligning til frekvensrespons
* z-transformasjon
    * Fourier og laplace (for kontekst)
    * Z-transformasjon
    * Z-planet
* Transferfunksjon $H(z)$

    

## Repetisjon IIR filtre

* **IIR** - **I**nfinite **I**mpulse **R**esponse
* Fellesbetegnelse for filter som kalkulerer utgangsveriden $y[n]$ basert på en lineær kombinasjon av verdiene til inngangssignalet ved foregående sampletidspunkt $x[n-k], \ \ k\in \{0, 1, 2, \ldots\}$ ***og*** tidligere kalkulerte verdier til utgangssignalet $y[n-k], \ \ k \in \{1, 2, \ldots\}$.


* **Differanseligning:**
$$\sum_{k=0}^{N} a_k \cdot y[n-k] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$
* **Filteralgoritme:**
$$y[n] = \frac{1}{a_0}\cdot \left(\sum_{k=0}^{M} b_k \cdot x[n-k] - \sum_{k=1}^{N}a_k\cdot  y[n-k] \right)$$

## Repetisjon Rekursiv filterstruktur

* IIR filtre omtales ofte som *rekursive* filtre ettersom utgangssignalet tilbakekobles som input.
<img src="Figurer/12_IIR%20Filtre/Fig0_Recursion.png" style="width: 35%; margin-left: 400px" />

* Impulsresponsen $h[n]$ for et slikt filter vil ha uendelig varighet.
* Impulsrespons og filterkoeffisienter er ikke lengre synonyme.
$$b_k \neq h[n]\big{|}_{n=k}$$

## Repetisjon IIR filtre

* Ved å lage et *rekursivt* filter, kan selv enkle førsteordens filtre produsere en langvarig impulsrespons.
* Et IIR-filter med relativt lav filterorden kan, dersom vi velger filterkoeffisienter med omhu, oppfylle de samme kravene for amplitduerespons som et FIR filter med ***mye*** høyere filterorden.
* Ulempen er at IIR filtre ***ikke*** har lineær faserespons i passbåndet. 
    * Konsekvensen av dette er *tidsforskyvningen* for hver av frekvenskomponentene til det filtrerte signalet gitt i antall sampler ikke vil være konstant.
    * Dette har potensiale til å forårsake uønsket distorsjon av f.eks. et lydsignal.

## Impulsrespons førsteordens IIR-filter

* Eksempel:
$$y[n] = x[n] + 0.9\cdot y[n-1]$$
* Ettersom et IIR-filter tar i bruk tidligere utgangsverdier får vi en *geometrisk rekke*:
|$$n$$|$$x[n]=\delta[n]$$|$$y[n]=h[n]$$|
|:-:|:-:|:-:|
|0|1|$$y[0] = 1 + 0.9\cdot 0 = 1$$|
|1|0|$$y[1] = 0 + 0.9\cdot 1 = 0.9$$|
|2|0|$$y[2] = 0 + 0.9\cdot 0.9 = 0.81$$|
|3|0|$$y[3] = 0 + 0.9\cdot 0.81 = 0.9^3$$|
|4|0|$$y[3] = 0 + 0.9\cdot 0.9^3 = 0.9^4$$|
|$$\ldots$$|$$\ldots$$|$$\ldots$$|

$$h[n] = u[n] \cdot 0.9^n$$

## Repetisjon Regneeksempel 3: 
* Gitt høypassfilteret
$$y[n] = x[n]-x[n-1]-0.8\cdot y[n-1]$$
Finn et uttrykk for frekvensresponsen, og korriger deretter filterkoeffisientene slik at $\text{max}\left(\left|H\left(e^{j\hat{\omega}}\right)\right|\right) = 1$.

In [ ]:
# Frekvensresponsen til filteret i eksempel 3
FreqRespDemo([0.1, -0.1], [1, 0.8], fig_num=1);

# Tidsforskyvning i frekvensplanet

* Vi vet at:
<br>


\begin{align}
DTFT(\alpha\cdot x[n-D]) &= \alpha \cdot  X\left(e^{j\hat{\omega}}\right) \cdot e^{-j\hat{\omega}\cdot D}\\
&\text{ettersom}\\
x[n-D] &= x[n]*\delta[n-D]\\
&\text{og}\\
DTFT(\delta[n-D]) &=e^{-j\hat{\omega}\cdot D}
\end{align}

* Dermed kan vi fouriertransformere begge sidene av differanseligningen til et filter.

# Alternativ metode: *DTFT av differanseligning*



\begin{align}
DTFT\left(\sum_{k=0}^{N} a_k \cdot y[n-k]\right) &= DTFT\left(\sum_{k=0}^{M} b_k \cdot x[n-k] \right)\\
\sum_{k=0}^{N} a_k \cdot Y\left(e^{j\hat{\omega}}\right)\cdot e^{-j\hat{\omega}\cdot k} &=\sum_{k=0}^{M} b_k \cdot X\left(e^{j\hat{\omega}}\right)\cdot e^{-j\hat{\omega}\cdot k}\\
Y\left(e^{j\hat{\omega}}\right)\cdot \left(\sum_{k=0}^{N} a_k \cdot e^{-j\hat{\omega}\cdot k}\right) &= X\left(e^{j\hat{\omega}}\right)\cdot \left(\sum_{k=0}^{M} b_k \cdot e^{-j\hat{\omega}\cdot k}\right)\\
H\left(e^{j\hat{\omega}}\right) = \frac{Y\left(e^{j\hat{\omega}}\right)}{X\left(e^{j\hat{\omega}}\right)} &= \frac{\sum_{k=0}^{M} b_k \cdot e^{-j\hat{\omega}\cdot k}}{\sum_{k=0}^{N} a_k \cdot e^{-j\hat{\omega}\cdot k}}
\end{align}

## Regneeksempel 1: 

* Gitt høypassfilteret
$$y[n] = x[n]-x[n-1]-0.8\cdot y[n-1]$$
Finn et uttrykk for frekvensresponsen  ved å fouriertransformere differanseligningen.

# Z-transformasjon

* Kartlegger filteregenskaper i "z-planet", som har 2 dimensjoner
* Viktig for analyse og design av digitale filtre
* Kan sammenlignes med Laplacetransformasjon for tidskontinuerlige LTI-system


## Oppsummering Fouriertransformasjon *(tidskontinuerlig)*

#### Formel

$$F(j\omega) = \int_0^{\infty}f(t)\cdot e^{-j\omega} dt$$

#### For LTI-system

* Dersom $f(t)$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x(t) = e^{j\omega \cdot t}$ med vinkelfrekvens $\omega$.
    * Gjelder også sinusformede signal da  $\cos(\omega\cdot t) = \frac{1}{2} e^{j\omega\cdot t} + \frac{1}{2} e^{-j \omega \cdot t}$.

# Laplacetransformasjon *(tidskontinuerlig)*

#### Formel:

\begin{align}
F(s) &= \int_0^{\infty}f(t)\cdot e^{-s\cdot t} dt\\
&\text{der}\\
s &= \sigma + j\omega
\end{align}

#### For LTI-system
* Dersom $f(t)$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x(t) = e^{-s \cdot t} = e^{-\sigma \cdot t} \cdot e^{-j \omega\cdot t}$.
* Refleksjoner:
    * Dersom $\sigma = 0$, reduseres den 2-dimensjonale laplacetrasformasjonen til fouriertransformasjon.
    * Dersom $\sigma > 0$, vil amplitudent til $x(t)$ synke over tid.
    * Dersom $\sigma < 0$, vil amplituden til $x(t)$ øke eksponentielt, og vi får "unbounded input".
    
<br>

In [21]:
# Angi punkt på s-planet
s = -1 + 2j*pi*8
# Regn ut e^(-st)
t = np.linspace(0, 1, 501)
x_t = exp(-s*t)

plt.close(2); plt.figure(2)
plt.subplot(2,1,1)
plt.plot(t, np.real(x_t))
plt.ylabel(r'Im($x(t)$)')
plt.grid(True)
plt.subplot(2,1,2)
plt.plot(t, np.imag(x_t))
plt.ylabel(r'Re($x(t)$)')
plt.xlabel(r't (sekund)')
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Hvorfor ikke bruke laplace for digitale filtre?

* Frekvenser $|\hat{\omega}|>\pi$ er vil ikke være gyldige.
* Aliasing må tas i betraktning.
    - Løses med å transformere til "polarkoordinater".
    - Digital frekvens $\hat{\omega}$ gitt ved vinkel
    - Demping gitt ved avstand til origo.

## Oppsummering DTFT
#### Formel

$$F\left(e^{j\hat{\omega}}\right) = \sum_{n=0}^{\infty}f[n]\cdot e^{-j\hat{\omega}\cdot n}$$

#### For LTI-system

* Dersom $f[n]$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x[n] = e^{-j\omega \cdot n}$ med vinkelfrekvens $\hat{\omega}$.
    * Gjelder også sinusformede signal da  $\cos(\hat{\omega}\cdot n) = \frac{1}{2} e^{j\hat{\omega}\cdot n} + \frac{1}{2} e^{-j \hat{\omega} \cdot n}$.

# Z-transformasjon
#### Notasjon:
$$\textbf{Z}(x[n]) = X(z)$$

#### Formel:
\begin{align}
H(z) &= \sum_{n=0}^{\infty} h[n]\cdot z^{-n}\\
& \text{der}\\
z &= r\cdot e^{j\hat{\omega}}
\end{align}

#### For LTI-system
* Dersom $h[n]$ er impulsresponsen til et LTI-system, forteller dette hvordan systemet vil påvirke et inngangssignal $x[n] = z^{-n} = r^{-n} \cdot e^{-j \hat{\omega}\cdot n}$.
* Refleksjoner:
    * Dersom $|r| = 1$, reduseres den 2-dimensjonale Z-transformen til DTFT. 
        * Dette er grunnen til at notasjonen for DTFT er $H\left(e^{j\hat{\omega}}\right)$.
    * Dersom $|r| < 1$, vil amplitudent til $x[n]$ synke over tid.
    * Dersom $|r| > 1$, vil amplituden til $x[n]$ øke eksponentielt, og vi får "unbounded input".
    
<br>

# S-plan og Z-plan
<br>
<img src="Figurer/14_Ztransform/Fig2_SandZ.png" style="width: 80%; margin-left: 100px" />

In [24]:
# Angi punkt på z-planet i polarkoordinater
z = 1.2*exp(2j*pi/8)

# Regn ut z^(-n)
n = np.arange(0, 32)
x_n = z**-n

plt.close(2); plt.figure(2)
plt.subplot(2,1,1)
plt.stem(n, np.real(x_n), markerfmt='.', basefmt='grey', use_line_collection=True)
plt.ylabel(r'Im($x[n]$)')
plt.grid(True)
plt.subplot(2,1,2)
plt.stem(n, np.imag(x_n), markerfmt='.', basefmt='grey', use_line_collection=True)
plt.ylabel(r'Re($x[n]$)')
plt.xlabel(r'n (samplenummer)')
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LTI-filter i Z-domene

<br>

<img src="Figurer/14_Ztransform/Fig3_TF.png" style="width: 80%; margin-left: 100px" />

# Transferfunksjon / overføringsfunksjon

* Z-transformen av impulsresponen $h[n]$ til et LTI-system kalles systemets *transferfunksjon*
* For ethvert kausalt LTI-system med filterkoeffisienter **a** og **b**, vil transferfunksjonen være:
\begin{align}
H(z) = \frac{Y(z)}{X(z)} &= \frac{\sum_{k=0}^{M} b_k \cdot z^{-k}}{\sum_{k=0}^{N} a_k \cdot z^{-k}}\\
&\text{eller}\\
&= \frac{b_0 + b_1\cdot z^{-1} + b_2 \cdot z^{-2} + \ldots + b_M \cdot z^{-M}}{a_0 + a_1\cdot z^{-1} + a_2 \cdot z^{-2} + \ldots + a_N \cdot z^{-N}}
\end{align}
* Teller og nevner er polynomer
* *samme utledning som for DTFT av differanseligning*

# Tidsforskyvning i Z-planet

* Hver sample tidsforskyvning tilsvarer multiplikasjon med $z^{-1}$ i z-planet. 
$$\textbf{Z}(x[n-D]) = z^{-D}\cdot X(z)$$

* I blokkskjemarepresentasjon av filter brukes derfor $z^{-1}$ for å symbolisere tidsforskyvning/delay.

<br>

<img src="Figurer/14_Ztransform/Fig4_Delay.png" style="width: 70%; margin-left: 100px" />

## Regneeksempel 2

* Finn transferfunksjonen $H(z)$ til filteret i vist i figuren:
<br>

<img src="Figurer/14_Ztransform/Fig5_ExFilt.png" style="width: 70%; margin-left: 100px" />

# Spørsmål?